# NSL-KDD Dataset Introduction

The NSL-KDD dataset was collated in order to mitigate the problems of the KDD'99 dataset. The KDD'99 dataset attracted the attention of many researchers due to the lack of public data sets for signature based intrusion detection systems (IDSs). Tavallaee et al. [1] found that the most significant issue with the 1999 dataset is that it contained a significant number of redundant instances such that classifiers were biased towards these instances while disregarding infrequent instances. However, it is often these infrequent instances that can be indicative of new devastating network intrusions.

Some of the improvements present in the NSL-KDD dataset is that all redundant instances were removed. Moreover, the number of records from each difficulty group was selected to be inversely proportional to the percentage of records in the original KDD dataset. Difficulty refers to how many of 21 classifiers were able to classify an instance correctly from the 1999 dataset. In addition, some of the instances in the original dataset contained unsystematic noise, thus these instances were discarded when constructing the improved dataset. Overall, the original KDD dataset was disproportionately distributed, thus rendering the classifiers ineffective in serving as a discriminative tool for detecting network intrusions that can be identified by signature.

The NSL-KDD dataset can be downloaded from: https://www.unb.ca/cic/datasets/nsl.html

Please note that a 20% subset of the whole dataset will be used due to the magnitude of the whole dataset (more than 120,000 instances). The 20% subset is also provided at the above link under the name 'KDDTrain+_20Percent.TXT'